## Lab 3: RADIO INTERFEROMETRY AT X BAND

In [3]:
import ugradio
import numpy as np
import matplotlib.pyplot as plt

### Week One
#### Rotation matrix for timing locations
Each group uses rotation matrices to calculate when objects of interest are up. 

#### Solar Fringes
Observe the Sun for a short time to confirm that it sees fringes. 

#### Observe a point source
Each group does a horizon-to-horizon observation of a point source, which requires writing an observing script to run automatically. 

#### Power spectra
Each person calculates Fourier power spectra of the Sun data and also the point-source data.

#### Compare to expected fringe frequencies
She/He calculates the range of expected local fringe frequencies (equation 14) and compares the observed spectra with these expectations. Be ready for show and tell!

### 8.1. Getting the Data
FIRST WEEK: Before doing the weak sources in the Table, do the Sun for a much shorter time, say an hour. This will give you confidence that the system works (or so we hope). There should be an easily-recognizable signal that you can look at visually, think about, and derive the
approximate declination with pencil and paper. Then later you can write software to do the same, and make sure you get the right answer. Also, during this first week, do the horizon-to-horizon track of one of the sources from the Table.

### 8.2. The Fringe

In [2]:
ugradio.timing.julian_date??